In [1]:
import pandas as pd
from collections import Counter

In [2]:
data = {'email':  ['Click Win Prize', 'Click meeting setup meeting','Prize free prize','Click prize free'],
        'is_spam': ['yes', 'no','yes','yes']
        }
data = pd.DataFrame(data)
data

,email,is_spam
0,Click Win Prize,yes
1,Click meeting setup meeting,no
2,Prize free prize,yes
3,Click prize free,yes


In [3]:
# make email values lower case
data["email"] = data["email"].apply(lambda str : str.lower()) 
data

,email,is_spam
0,click win prize,yes
1,click meeting setup meeting,no
2,prize free prize,yes
3,click prize free,yes


In [4]:
#get probability of spam and not spam
sum_s = 0
sum_ns = 0
for i in data["is_spam"]:
    if i == "yes":
        sum_s += 1
    else:
        sum_ns += 1
        
prob_s = sum_s/len(data)
prob_ns = sum_ns/len(data)

print(prob_s)
print(prob_ns)

0.75
0.25


In [12]:
data["bow"] = data["email"].str.split().apply(Counter) # split string into the bag of words an then count them
data

,email,is_spam,bow
0,click win prize,yes,"{'click': 1, 'win': 1, 'prize': 1}"
1,click meeting setup meeting,no,"{'click': 1, 'meeting': 2, 'setup': 1}"
2,prize free prize,yes,"{'prize': 2, 'free': 1}"
3,click prize free,yes,"{'click': 1, 'prize': 1, 'free': 1}"


In [38]:
sum_bow_s = {}
sum_bow_ns = {}
i=0

for bow in data["bow"]: #value of bag of word in each email
    for key in bow: #value of word in each bag of word
        if data["is_spam"][i] == "yes": #in case of spam
            if key in sum_bow_s: #if the word exists in sum of bows
                # print("sum of bow in if: ", sum_bow_s)
                sum_bow_s[key] += bow[key]
                # print("sum of bow in if: ", sum_bow_s)

            else:
                # print("key in else: ",key)
                # print('bow of key', bow[key])
                sum_bow_s[key] = bow[key]
                # print("sum of bow in else: ", sum_bow_s)
        else:
            if key in sum_bow_ns:
                sum_bow_ns[key] +=bow[key]
            else:
                sum_bow_ns[key] = bow[key]
    i += 1


print(sum_bow_s)
print(sum_bow_ns)

{'click': 2, 'win': 1, 'prize': 4, 'free': 2}
{'click': 1, 'meeting': 2, 'setup': 1}


In [7]:
#calculates total number of words in each dictionary
term_occurences_s = 0
term_occurences_ns = 0
for key in sum_bow_s:
    term_occurences_s += sum_bow_s[key]
for key in sum_bow_ns:    
    term_occurences_ns += sum_bow_ns[key]
print(term_occurences_s)
print(term_occurences_ns)

9
4


In [8]:
#calculate number of unique words in total
temp = []
unique_term = 0
for bow in data["bow"]:
    for key in bow:
        if key not in temp:
            unique_term += 1
            temp.append(key)
# it goes to every single word in BoW and if it is not stored in the array store it and increase unique_term number by 1
unique_term

6

In [39]:
sum_bow = {} 
term_occurences = 0 #number of words occured in total in spam and not spam
for bow in data["bow"]:
    for key in bow:
        if key in sum_bow:
            sum_bow[key] += bow[key]
        else:
            sum_bow[key] = bow[key]
        term_occurences += bow[key]


prob_bow = {}
for key in sum_bow:
    prob_bow[key] = sum_bow[key]/term_occurences #probability of word in general for spam and not spam

prob_bow  

13


{'click': 0.23076923076923078,
 'win': 0.07692307692307693,
 'prize': 0.3076923076923077,
 'meeting': 0.15384615384615385,
 'setup': 0.07692307692307693,
 'free': 0.15384615384615385}

In [65]:
def split(words): #split string into array of words
    return words.split()


def lower(words): #lowercase the letters in the string
    return words.lower()

def probSpam(email): #calculate the probability of spam of the string
    email = split(lower(email))
    likelihood = 1 
    evidence = 1
    for term in email:
        if term in prob_bow: #check if word has probability value so that it won't throw error for unrecognised words
            if term in sum_bow_s:
                likelihood *= (sum_bow_s[term]+1)/(term_occurences_s+unique_term) #naive beyes formula 
            else:
                likelihood *= 1/(term_occurences_s+unique_term)

            evidence *= prob_bow[term]

    likelihood_prior = likelihood*prob_s
    posterior = likelihood_prior/evidence
    return posterior

def probNotSpam(email): #calculate the probability of not spam of the string
    email = split(lower(email))
    likelihood = 1
    evidence = 1
    for term in email:
        if term in prob_bow:
            if term in sum_bow_ns:
                likelihood *= (sum_bow_ns[term]+1)/(term_occurences_ns+unique_term)
            else:
                likelihood *= 1/(term_occurences_ns+unique_term)
                
            evidence *= prob_bow[term]

    likelihood_prior = likelihood*prob_ns
    posterior = likelihood_prior/evidence
    return posterior


print(probSpam("Free setup meeting free game awgawg5 dfgw4 aw 34324 "))
print(probNotSpam("Free setup meeting free 325rt23 rtrdfgWE WER WER "))

0.4760166666666666
0.53551875
